# Silver Transformation Engine
Universal engine for cleaning, trimming, and applying table-specific business logic.

In [0]:
%python
from pyspark.sql import functions as F
from pyspark.sql.types import *

def run_silver_pipeline(source_table, target_table, custom_logic_func=None, rename_map=None):
    """
    Orchestrates the silver transformation process including universal trimming,
    custom business logic, and column renaming.
    """
    # 1. Read from Bronze
    df = spark.table(f"workspace.bronze.{source_table}")
    
    # 2. Universal Cleaning (Trimming all string columns)
    for field in df.schema.fields:
        if isinstance(field.dataType, StringType):
            df = df.withColumn(field.name, F.trim(F.col(field.name)))
            
    # 3. Apply Table-Specific Business Logic 
    if custom_logic_func:
        df = custom_logic_func(df)
        
    # 4. Apply Column Renaming 
    if rename_map:
        for old_name, new_name in rename_map.items():
            df = df.withColumnRenamed(old_name, new_name)
            
    # 5. Add Metadata and Save to Silver Layer
    df = df.withColumn("silver_ingestion_ts", F.current_timestamp())
    df.write.mode("overwrite").format("delta").saveAsTable(f"workspace.silver.{target_table}")
    
    print(f"✅ Table {target_table} created successfully!")